In [1]:
import json
import requests
import itertools
from collections import Counter, OrderedDict
import pandas as pd
%matplotlib inline
from tqdm import tqdm

In [2]:
file_path = '/nfs/gns/literature/machine-learning/evaluation/20karticles/europePMC-NER/annotations_API/full_sentences/tagged_sentences/Europe_PMC_annotation.csv'
annot_csv = pd.read_csv(file_path, names=['pmc_id', 'section', 'sentence', 'ner'],sep='\t')

In [3]:
import os
import pandas as pd
import glob
import json
import csv
import sys

import multiprocessing

import numpy as np
import re

from nltk.tokenize import wordpunct_tokenize

import requests
# from pprint import pprint
import pandas as pd

from collections import defaultdict, Counter
import time
from requests.compat import urljoin

from tqdm import tqdm

In [4]:
from flair.models import TextClassifier
from flair.data import Sentence, Token
from flair.models import SequenceTagger

flair_models = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/manual_annotated_dataset/'
# load the model you trained
model = SequenceTagger.load(flair_models+'best-model.pt')



2020-01-28 14:22:34,509 loading file /nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/manual_annotated_dataset/best-model.pt


In [5]:
# # for multi processing
# import numpy as np
# count = 0
# for g, df in annot_csv.groupby(np.arange(len(annot_csv)) // 16):
#     count = count+1
#     list_sent = []
#     print(df.shape)
#     for index_, each_annotation in enumerate(df.itertuples(), 0):
#         print(index_,each_annotation.sentence)
#     if count ==1:
#         break

In [6]:
annot_csv.columns

Index(['pmc_id', 'section', 'sentence', 'ner'], dtype='object')

In [7]:
len(annot_csv)

1893974

In [8]:
from ast import literal_eval
from tqdm import tqdm

result_path = '/nfs/gns/literature/machine-learning/evaluation/20K-ML-NER/'
result_file_name = '20K-DS-Data.csv'

with open(result_path + result_file_name, 'a', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index_, each_annotation in tqdm(enumerate(annot_csv.itertuples()),total=len(annot_csv)):
#         if index_ <10:
        annot = []
        pmc_id = each_annotation.pmc_id
        section = each_annotation.section
        epmc_sentence = each_annotation.sentence

        for each_epmc_ner in literal_eval(each_annotation.ner):
            if each_epmc_ner[2] == 'Diseases':
                annot.append(each_epmc_ner[1])

        if annot:
            new_row = [pmc_id, section, epmc_sentence, annot]
            public_writer.writerows([new_row])
                


100%|██████████| 1893974/1893974 [00:44<00:00, 42255.52it/s]


In [9]:
from nltk.tokenize import wordpunct_tokenize, WordPunctTokenizer

def custom_tokenizer(text):
    """
    Tokenizer based on word and punctuations only.
    """
    tokens: List[Token] = []

    tokenizer = WordPunctTokenizer()

    text = tokenizer.tokenize(text)

    index = 0
    for index, word in enumerate(text):
        tokens.append(
            Token(
                text=word, start_position=index, whitespace_after=False
            )
        )

    return tokens



In [10]:
DS_annot_csv = pd.read_csv(result_path + result_file_name, names=['pmc_id', 'section', 'sentence', 'ner'],sep='\t')

In [13]:
DS_annot_csv_ = DS_annot_csv.head(32)
# DS_annot_csv_

In [ ]:
# import numpy as np
# count = 0
# for g, df in annot_csv.groupby(np.arange(len(annot_csv)) // 16):
#     count = count+1
#     list_sent = []
#     print(df.shape)
#     for index_, each_annotation in enumerate(df.itertuples(), 0):
#         print(index_,each_annotation.sentence)
#     if count ==1:
#         break

In [16]:
result_path = '/nfs/gns/literature/machine-learning/evaluation/20K-ML-NER/'
result_file_name = '20K-DS_PCSE-Annotations.csv'

DS_data_annot = result_path + result_file_name

with open(DS_data_annot, 'w', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for g, anno_df in tqdm(DS_annot_csv.groupby(np.arange(len(DS_annot_csv)) // 16)):
        
        epmc_sentence = []
        pmc_id = []
        section = []
        epmc_ner = []
        sentences = []
        
#         for index_, each_annotation in tqdm(enumerate(anno_df.itertuples()),total=len(anno_df)):
        for each_annotation in anno_df.itertuples():
            pmc_id.append(each_annotation.pmc_id)
            section.append(each_annotation.section)
            epmc_sentence.append(each_annotation.sentence)
            epmc_ner.append(each_annotation.ner)
            sentences.append(Sentence(each_annotation.sentence, use_tokenizer=custom_tokenizer)) 

        predicted_sentences = model.predict(sentences, mini_batch_size=32)


        for i in range(0,len(epmc_sentence)):
            entities = predicted_sentences[i].to_dict(tag_type='ner')['entities']
            all_entities = []    
            if entities:
                tagged_sents = predicted_sentences[i].to_dict(tag_type='ner')
                for root_node in tagged_sents['entities']:
                    exact = root_node['text']
                    entity = root_node['type']

                    if entity == 'DS':    
                        all_entities.append(exact)              

            predicted_ner = [pmc_id[i], section[i], epmc_sentence[i], epmc_ner[i], all_entities]
            public_writer.writerow(predicted_ner)

        

100%|██████████| 30763/30763 [6:53:34<00:00,  1.24it/s]   


# Analysis

In [17]:
from ast import literal_eval


file_20k_path = '/nfs/gns/literature/machine-learning/evaluation/20K-ML-NER/'
file_20k_name = file_20k_path+'20K-DS_PCSE-Annotations.csv'


file_20k_df = pd.read_csv(file_20k_name, names=['pmc_id', 'section', 'sentence', 'epmc_ner','ml_ner'],sep='\t')




In [18]:
# Remove duplicate entries of the GP from the CSV
updated_epmc_ner_non_duplicates = []

for index_, each_annotation in tqdm(enumerate(file_20k_df.itertuples()),total=len(file_20k_df)):
    epmc_ner = literal_eval(each_annotation.epmc_ner)
    epmc_sentence = each_annotation.sentence
    new_epmc_ner = []
    for each_epmc_ner in set(epmc_ner):
        if epmc_ner.count(each_epmc_ner) > 1:
            count = epmc_sentence.count(each_epmc_ner)
            new_epmc_ner.extend([each_epmc_ner]*count)
        else:
            new_epmc_ner.append(each_epmc_ner)
#         print(new_epmc_ner)
    updated_epmc_ner_non_duplicates.append(new_epmc_ner)

100%|██████████| 492195/492195 [00:12<00:00, 38073.45it/s]


In [19]:
file_20k_df['eurpmc_ner'] =  updated_epmc_ner_non_duplicates
file_20k_df.head(15)

,pmc_id,section,sentence,epmc_ner,ml_ner,eurpmc_ner
0,PMC4782685,DISCUSS,"For instance, cancer registries or retrospecti...",['cancer'],['cancer'],[cancer]
1,PMC4610596,INTRO,It has been implicated in fundamental function...,['neurological disorders'],['chronic neurological disorders'],[neurological disorders]
2,PMC4610596,INTRO,WPT has been extensively studied for implantab...,['gastro-esophageal reflux disease'],"['gastro-esophageal reflux disease', 'GERD']",[gastro-esophageal reflux disease]
3,PMC4610596,UNK,The flexible polyimide substrate with a thickn...,['scars'],[],[scars]
4,PMC4726007,RESULTS,"To further test the algorithm’s performance, w...",['bacteremia'],['bacteremia'],[bacteremia]
5,PMC4726007,DISCUSS,This enables our LAM technique to scale to the...,['polymicrobial infection'],['polymicrobial infection'],[polymicrobial infection]
6,PMC5712973,METHODS,"In more detail, the radial distortion is mainl...",['defect'],[],[defect]
7,PMC4083040,DISCUSS,This idea was corroborated by our results and ...,['addiction'],[],[addiction]
8,PMC4083040,DISCUSS,The case of tobacco addiction is a particular ...,['addiction'],[],[addiction]
9,PMC4083040,DISCUSS,"On the other hand, smokers can be against new ...",['addiction'],[],[addiction]


In [20]:
k_exacts = []
k_preds = []
k_missing = []


for index_, each_annotation in tqdm(enumerate(file_20k_df.itertuples()),total=len(file_20k_df)):
#     if index_ <10:
    #     pmc_id = each_annotation.pmc_id
    #     section = each_annotation.section
    #     epmc_sentence = each_annotation.sentence
#         k_exacts.append(literal_eval(each_annotation.eurpmc_ner))
        k_exacts.append(each_annotation.eurpmc_ner)
        k_preds.append(literal_eval(each_annotation.ml_ner))

    

100%|██████████| 492195/492195 [00:11<00:00, 43470.73it/s]


In [21]:
def no_intersection(lst1, lst2): 
    miss = []
    temp = set(lst2) 
    for value in lst1:
        scores = [fuzz.partial_ratio(value,sublist) for sublist in temp]
        if 100 not in scores:
#         if any(i < 70for i in scores):
            miss.append(value)  
    return miss
    

In [22]:
from fuzzywuzzy import fuzz

In [23]:
k_missing = []
for i,j in tqdm(zip(k_exacts,k_preds), total = len(k_exacts)):
    if i:
        missing = no_intersection(i, j)
        if missing:
            k_missing.append(missing)

100%|██████████| 492195/492195 [00:19<00:00, 25000.12it/s]


In [24]:
missing_data = list(itertools.chain.from_iterable(k_missing))
count = Counter(missing_data)
y = OrderedDict(count.most_common())

In [25]:
import pickle

path = '/nfs/gns/literature/machine-learning/Santosh/FP_analysis/20K/'
with open(path+'DS_k_exacts', 'wb') as fp:
    pickle.dump(k_exacts, fp)
    
with open(path+'DS_k_preds', 'wb') as fp:
    pickle.dump(k_preds, fp) 

In [26]:
anno_data = list(itertools.chain.from_iterable(k_exacts))
anno_count = Counter(anno_data)
anno_y = OrderedDict(anno_count.most_common())

In [27]:
import csv

with open(path+"anno_non_dup_frequencies_DS_20K.csv", "w",  newline='\n') as outfile:
    public_writer = csv.writer(outfile, delimiter='\t', lineterminator='\n')
    for keys, values in y.items(): # Get keys from the missing
        filt_per = (values/anno_y[keys])*100
        new_row = [keys, str(values), str(anno_y[keys]), str(filt_per)]
        public_writer.writerows([new_row])

In [89]:
## Convert to json

In [28]:
# file_20k_df['eurpmc_ner'] = file_20k_df.eurpmc_ner.apply(lambda x: literal_eval(str(x)))
file_20k_df['ml_ner'] = file_20k_df.ml_ner.apply(lambda x: literal_eval(str(x)))

In [29]:
interested_cloumns = ['pmc_id', 'section', 'sentence', 'eurpmc_ner','ml_ner']
file_20k_df[interested_cloumns].head(15)


,pmc_id,section,sentence,eurpmc_ner,ml_ner
0,PMC4782685,DISCUSS,"For instance, cancer registries or retrospecti...",[cancer],[cancer]
1,PMC4610596,INTRO,It has been implicated in fundamental function...,[neurological disorders],[chronic neurological disorders]
2,PMC4610596,INTRO,WPT has been extensively studied for implantab...,[gastro-esophageal reflux disease],"[gastro-esophageal reflux disease, GERD]"
3,PMC4610596,UNK,The flexible polyimide substrate with a thickn...,[scars],[]
4,PMC4726007,RESULTS,"To further test the algorithm’s performance, w...",[bacteremia],[bacteremia]
5,PMC4726007,DISCUSS,This enables our LAM technique to scale to the...,[polymicrobial infection],[polymicrobial infection]
6,PMC5712973,METHODS,"In more detail, the radial distortion is mainl...",[defect],[]
7,PMC4083040,DISCUSS,This idea was corroborated by our results and ...,[addiction],[]
8,PMC4083040,DISCUSS,The case of tobacco addiction is a particular ...,[addiction],[]
9,PMC4083040,DISCUSS,"On the other hand, smokers can be against new ...",[addiction],[]


In [30]:
jsonFilePath = 'DS_file_20k_df_non_dup.json'
file_20k_df[interested_cloumns].to_json(path+jsonFilePath, orient='records')

In [9]:
import pandas as pd
path = '/nfs/gns/literature/machine-learning/Santosh/FP_analysis/20K/'
complete_path = path+'anno_non_dup_frequencies_DS_20K.csv'
columns_freq = ['Entity', 'Freq_of_ML_removed','EPMC_total_Freq','Percentage_removed']
anno_freq_df = pd.read_csv(complete_path, sep = '\t', error_bad_lines=False, names = columns_freq)

jsonFilePath = 'EPMC_DS_freqs.json'
anno_freq_df.to_json(path+jsonFilePath, orient='records')
